# Baseline Model using Opcode Frequency for Binary

## Set up

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import os
import json
import pandas as pd
from imblearn.combine import SMOTETomek
import numpy as np
from pathlib import Path

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score


In [3]:
PATH = Path.cwd().parents[2]
DATA_PATH = os.path.join(PATH, 'data/processed/opcode_freq')

## List Traditional ML model to compare

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, ExtraTreesClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.neural_network import MLPClassifier

models = {
  "LogisticRegression()": LogisticRegression(),
  "DecisionTreeClassifier()": DecisionTreeClassifier(),
  "RandomForestClassifier()": RandomForestClassifier(),
  "AdaBoostClassifier()": AdaBoostClassifier(),
  "ExtraTreesClassifier()": ExtraTreesClassifier(),
  "XGBClassifier()": XGBClassifier(),
  "LGBMClassifier()": LGBMClassifier(),
  "SVC()": SVC(),
  "GaussianNB()": GaussianNB(),
  "KNeighborsClassifier()": KNeighborsClassifier(),
  "SGDClassifier()": SGDClassifier(),
  "MLPClassifier()": MLPClassifier(),
}

## Load Dataset

In [5]:
df = pd.read_csv(os.path.join(DATA_PATH, 'dataset.csv')).set_index('address')

with open(os.path.join(DATA_PATH, 'features.json'), "r") as f:
    features = json.load(f)

with open(os.path.join(DATA_PATH, 'labels.json'), "r") as f:
    labels = json.load(f)

In [6]:
df.head()

,mint,leak,limit,PUSH,MSTORE,CALLDATASIZE,LT,JUMPI,CALLDATALOAD,SWAP,...,MULMOD,DIFFICULTY,BALANCE,BASEFEE,SDIV,CALLCODE,SGT,EXTCODECOPY,SIGNEXTEND,DELEGATECALL
address,,,,,,,,,,,,,,,,,,,,,
0x93023f1d3525e273f291b6f76d2f5027a39bf302,1,0,1,734,119.0,14,9.0,67,8.0,187,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0x2753dce37a7edb052a77832039bcc9aa49ad8b25,0,0,1,1155,179.0,23,13.0,101,7.0,264,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0x94b7d24552933f50a5a5705c446528806dcea381,0,0,0,20,3.0,2,0.0,2,1.0,4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
0xe0b9d4146aad6936cbfcbe4dae47e34aab96b093,0,0,0,1135,107.0,9,10.0,96,3.0,464,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0x10f6f2b97f3ab29583d9d38babf2994df7220c21,1,0,1,1623,150.0,18,18.0,115,6.0,722,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
features[:5]

['PUSH', 'MSTORE', 'CALLDATASIZE', 'LT', 'JUMPI']

In [8]:
labels[:5]

['mint', 'leak', 'limit']

## Prepare Features and Labels

In [9]:
X = df[features]
y_mint = df[['mint']]
y_leak = df[['leak']]
y_limit = df[['limit']]

## Train/Test Split

In [10]:
X_mint_train, X_mint_test, y_mint_train, y_mint_test = train_test_split(
    X, y_mint, test_size=0.2, random_state=42
)

X_leak_train, X_leak_test, y_leak_train, y_leak_test = train_test_split(
    X, y_leak, test_size=0.2, random_state=42
)

X_limit_train, X_limit_test, y_limit_train, y_limit_test = train_test_split(
    X, y_limit, test_size=0.2, random_state=42
)

## Run all models and collect reports

In [11]:
def get_report_all_ml(X_train, y_train, X_test, y_test):
    report_list = []

    for name, model in models.items():
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Generate classification report (as dict)
        report_dict = classification_report(y_test, y_pred, output_dict=True, zero_division=0)

        # Average scores across all labels (macro average)
        avg_scores = report_dict["macro avg"]

        report_list.append({
            "Model": name,
            "Precision": avg_scores["precision"],
            "Recall": avg_scores["recall"],
            "F1-score": avg_scores["f1-score"]
        })

    df_report = pd.DataFrame(report_list)
    df_report = df_report.sort_values("F1-score", ascending=False).reset_index(drop=True)

    return df_report

In [12]:
def display_cm(y_test, y_pred):
    cm = confusion_matrix(y_test, y_pred)
    cm_display = ConfusionMatrixDisplay(confusion_matrix = cm, display_labels = [0, 1])
    return cm_display

## Show report

In [13]:
data = {
  'mint': {
    "X_train": X_mint_train,
    "X_test": X_mint_test,
    "y_train": y_mint_train,
    "y_test": y_mint_test
  },
  'leak': {
    "X_train": X_leak_train,
    "X_test": X_leak_test,
    "y_train": y_leak_train,
    "y_test": y_leak_test
  },
  'limit': {
    "X_train": X_limit_train,
    "X_test": X_limit_test,
    "y_train": y_limit_train,
    "y_test": y_limit_test
  }
}

reports = []

for d in data.values():
    X_train = d['X_train']
    X_test = d['X_test']
    y_train = d['y_train']
    y_test = d['y_test']
    df_report = get_report_all_ml(X_train, y_train, X_test, y_test)
    reports.append(df_report)

[LightGBM] [Info] Number of positive: 14, number of negative: 41
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000155 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 634
[LightGBM] [Info] Number of data points in the train set: 55, number of used features: 51
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.254545 -> initscore=-1.074515
[LightGBM] [Info] Start training from score -1.074515
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best ga

In [14]:
reports[0]

,Model,Precision,Recall,F1-score
0,DecisionTreeClassifier(),0.863636,0.750000,0.754386
1,KNeighborsClassifier(),0.863636,0.750000,0.754386
2,ExtraTreesClassifier(),0.833333,0.666667,0.650000
3,AdaBoostClassifier(),0.833333,0.666667,0.650000
4,MLPClassifier(),0.833333,0.666667,0.650000
5,LGBMClassifier(),0.833333,0.666667,0.650000
6,XGBClassifier(),0.807692,0.583333,0.523810
7,RandomForestClassifier(),0.807692,0.583333,0.523810
8,GaussianNB(),0.500000,0.500000,0.497436
9,LogisticRegression(),0.541667,0.520833,0.475000


In [15]:
reports[1]

,Model,Precision,Recall,F1-score
0,KNeighborsClassifier(),0.923077,0.666667,0.708333
1,AdaBoostClassifier(),0.666667,0.621212,0.634783
2,GaussianNB(),0.488889,0.484848,0.475000
3,SVC(),0.392857,0.500000,0.440000
4,SGDClassifier(),0.392857,0.500000,0.440000
5,MLPClassifier(),0.392857,0.500000,0.440000
6,LGBMClassifier(),0.392857,0.500000,0.440000
7,LogisticRegression(),0.384615,0.454545,0.416667
8,ExtraTreesClassifier(),0.384615,0.454545,0.416667
9,RandomForestClassifier(),0.384615,0.454545,0.416667


In [16]:
reports[2]

,Model,Precision,Recall,F1-score
0,LogisticRegression(),0.666667,0.677778,0.641026
1,DecisionTreeClassifier(),0.666667,0.677778,0.641026
2,ExtraTreesClassifier(),0.666667,0.677778,0.641026
3,MLPClassifier(),0.625000,0.633333,0.625668
4,LGBMClassifier(),0.571429,0.577778,0.562500
5,RandomForestClassifier(),0.571429,0.577778,0.562500
6,XGBClassifier(),0.571429,0.577778,0.562500
7,AdaBoostClassifier(),0.520833,0.522222,0.497436
8,GaussianNB(),0.484848,0.488889,0.475000
9,KNeighborsClassifier(),0.425000,0.433333,0.426901


## MLP

### Build model

In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, LeakyReLU, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.losses import MeanSquaredError

2025-07-15 18:21:31.841766: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [18]:
def Model(input_dim, output_dim):
    model = Sequential([
        Input(shape=(input_dim,)),
        Dense(512, kernel_regularizer=regularizers.l1_l2(1e-6)),
        BatchNormalization(),
        LeakyReLU(0.01),
        Dropout(0.4),

        Dense(256, kernel_regularizer=regularizers.l1_l2(1e-6)),
        BatchNormalization(),
        LeakyReLU(0.01),
        Dropout(0.3),

        Dense(128, kernel_regularizer=regularizers.l1_l2(1e-6)),
        BatchNormalization(),
        LeakyReLU(0.01),
        Dropout(0.2),

        Dense(output_dim, activation='sigmoid')  # sigmoid for multi-label
    ])

    model.compile(
        loss='binary_crossentropy',
        optimizer=Adam(learning_rate=1e-6),
        metrics=['accuracy']
    )
    return model

In [19]:
model_mint = Model(input_dim=X.shape[1], output_dim=1)
model_leak = Model(input_dim=X.shape[1], output_dim=1)
model_limit = Model(input_dim=X.shape[1], output_dim=1)

### Train

In [20]:
model_mint.fit(X_mint_train, y_mint_train, validation_split=0.2, epochs=100, batch_size=32,
                    callbacks=[
                                EarlyStopping(monitor='val_loss',
                                             patience=5,
                                             restore_best_weights=True),
                                ReduceLROnPlateau(
                                  monitor='val_loss',
                                  factor=0.5,
                                  patience=5,
                                  verbose=1)
                               ]
                    )

Epoch 1/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 300ms/step - accuracy: 0.2756 - loss: 1.2150 - val_accuracy: 0.1818 - val_loss: 9.3182 - learning_rate: 1.0000e-06
Epoch 2/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - accuracy: 0.3466 - loss: 1.1368 - val_accuracy: 0.1818 - val_loss: 6.6817 - learning_rate: 1.0000e-06
Epoch 3/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 0.3674 - loss: 1.1762 - val_accuracy: 0.1818 - val_loss: 5.1496 - learning_rate: 1.0000e-06
Epoch 4/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - accuracy: 0.3419 - loss: 1.0773 - val_accuracy: 0.1818 - val_loss: 4.4179 - learning_rate: 1.0000e-06
Epoch 5/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - accuracy: 0.2652 - loss: 1.0827 - val_accuracy: 0.1818 - val_loss: 4.0066 - learning_rate: 1.0000e-06
Epoch 6/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - accuracy: 0.4290 - loss: 1.1019 - val_accuracy: 0.1818 - val_loss: 3.6116 - learning_rate: 1.0000e-06
Epoch 7/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 0.3826 - loss

In [21]:
model_leak.fit(X_leak_train, y_leak_train, validation_split=0.2, epochs=100, batch_size=32,
                    callbacks=[
                                EarlyStopping(monitor='val_loss',
                                             patience=5,
                                             restore_best_weights=True),
                                ReduceLROnPlateau(
                                  monitor='val_loss',
                                  factor=0.5,
                                  patience=5,
                                  verbose=1)
                               ]
                    )

Epoch 1/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 291ms/step - accuracy: 0.1733 - loss: 1.0877 - val_accuracy: 0.4545 - val_loss: 2.3066 - learning_rate: 1.0000e-06
Epoch 2/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 0.2860 - loss: 1.0450 - val_accuracy: 0.4545 - val_loss: 1.8015 - learning_rate: 1.0000e-06
Epoch 3/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - accuracy: 0.2348 - loss: 0.9996 - val_accuracy: 0.4545 - val_loss: 1.5743 - learning_rate: 1.0000e-06
Epoch 4/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - accuracy: 0.2453 - loss: 1.0487 - val_accuracy: 0.4545 - val_loss: 1.4340 - learning_rate: 1.0000e-06
Epoch 5/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - accuracy: 0.3011 - loss: 1.0135 - val_accuracy: 0.4545 - val_loss: 1.3395 - learning_rate: 1.0000e-06
Epoch 6/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 0.3163 - loss: 1.1497 - val_accuracy: 0.4545 - val_loss: 1.2804 - learning_rate: 1.0000e-06
Epoch 7/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 0.1884 - loss

In [22]:
model_limit.fit(X_limit_train, y_limit_train, validation_split=0.2, epochs=100, batch_size=32,
                    callbacks=[
                                EarlyStopping(monitor='val_loss',
                                             patience=5,
                                             restore_best_weights=True),
                                ReduceLROnPlateau(
                                  monitor='val_loss',
                                  factor=0.5,
                                  patience=5,
                                  verbose=1)
                               ]
                    )

Epoch 1/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 294ms/step - accuracy: 0.5152 - loss: 0.6905 - val_accuracy: 0.2727 - val_loss: 3.9154 - learning_rate: 1.0000e-06
Epoch 2/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 199ms/step - accuracy: 0.5814 - loss: 0.7418 - val_accuracy: 0.3636 - val_loss: 2.8249 - learning_rate: 1.0000e-06
Epoch 3/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - accuracy: 0.5407 - loss: 0.7294 - val_accuracy: 0.3636 - val_loss: 2.3087 - learning_rate: 1.0000e-06
Epoch 4/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - accuracy: 0.5208 - loss: 0.8015 - val_accuracy: 0.3636 - val_loss: 2.0404 - learning_rate: 1.0000e-06
Epoch 5/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - accuracy: 0.5256 - loss: 0.7309 - val_accuracy: 0.3636 - val_loss: 1.8373 - learning_rate: 1.0000e-06
Epoch 6/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - accuracy: 0.6278 - loss: 0.6307 - val_accuracy: 0.3636 - val_loss: 1.6874 - learning_rate: 1.0000e-06
Epoch 7/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - accuracy: 0.6023 - los

### Predict

In [23]:
def tune_thresholds(y_true, y_pred_prob, metric='f1'):
    y_true = np.asarray(y_true)          # Fix: convert to NumPy
    y_pred_prob = np.asarray(y_pred_prob)

    best_thresholds = []
    best_scores = []

    for i in range(y_true.shape[1]):
        label_true = y_true[:, i]
        label_probs = y_pred_prob[:, i]  # Fix here too

        thresholds = np.linspace(0.0, 1.0, 101)
        scores = []

        for t in thresholds:
            label_pred = (label_probs >= t).astype(int)
            if metric == 'f1':
                score = f1_score(label_true, label_pred, zero_division=0)
            scores.append(score)

        best_t = thresholds[np.argmax(scores)]
        best_score = np.max(scores)

        best_thresholds.append(best_t)
        best_scores.append(best_score)

        print(f"Label {i}: Best threshold = {best_t:.2f}, Best {metric} = {best_score:.4f}")

    return best_thresholds, best_scores


In [24]:
y_mint_test_prob = model_mint.predict(X_mint_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 162ms/step


In [25]:
best_thresholds, _ = tune_thresholds(y_mint_test, y_mint_test_prob)

Label 0: Best threshold = 0.54, Best f1 = 0.6667


In [26]:
y_mint_pred = (model_mint.predict(X_mint_test) >= best_thresholds).astype(int)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step


In [27]:
print(classification_report(y_mint_test, y_mint_pred))

              precision    recall  f1-score   support

           0       1.00      0.25      0.40         8
           1       0.50      1.00      0.67         6

    accuracy                           0.57        14
   macro avg       0.75      0.62      0.53        14
weighted avg       0.79      0.57      0.51        14



In [28]:
y_leak_test_prob = model_leak.predict(X_leak_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step


In [29]:
best_thresholds, _ = tune_thresholds(y_leak_test, y_leak_test_prob)

Label 0: Best threshold = 0.63, Best f1 = 0.6667


In [30]:
y_leak_pred = (model_leak.predict(X_leak_test) >= best_thresholds).astype(int)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step


In [31]:
print(classification_report(y_leak_test, y_leak_pred))

              precision    recall  f1-score   support

           0       0.91      0.91      0.91        11
           1       0.67      0.67      0.67         3

    accuracy                           0.86        14
   macro avg       0.79      0.79      0.79        14
weighted avg       0.86      0.86      0.86        14



In [32]:
y_limit_test_prob = model_limit.predict(X_limit_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step


In [33]:
best_thresholds, _ = tune_thresholds(y_limit_test, y_limit_test_prob)

Label 0: Best threshold = 0.00, Best f1 = 0.7826


In [34]:
y_limit_pred = (model_leak.predict(X_limit_test) >= best_thresholds).astype(int)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step


In [35]:
print(classification_report(y_limit_test, y_limit_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         5
           1       0.64      1.00      0.78         9

    accuracy                           0.64        14
   macro avg       0.32      0.50      0.39        14
weighted avg       0.41      0.64      0.50        14



## Autoencoder + MPL

In [36]:
def Autoencoder(input_dim=256):
    model = Sequential([
        Input(shape=(input_dim,)),
        Dense(128),
        LeakyReLU(0.01),

        Dense(32),
        BatchNormalization(),
        LeakyReLU(0.01),

        Dense(128),
        BatchNormalization(),
        LeakyReLU(0.01),

        Dense(input_dim, activation='sigmoid')  # sigmoid for multi-label
    ])

    model.compile(optimizer=Adam(1e-6), loss=MeanSquaredError())
    return model


In [37]:
autoencoder = Autoencoder(input_dim=X.shape[1])
autoencoder.fit(X_mint_train, X_mint_train, epochs=50, batch_size=32, validation_split=0.2)

Epoch 1/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 216ms/step - loss: 39988.2148 - val_loss: 49044.2148
Epoch 2/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - loss: 42187.2617 - val_loss: 49044.6250
Epoch 3/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - loss: 36486.2305 - val_loss: 49045.3906
Epoch 4/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - loss: 41967.4883 - val_loss: 49045.8008
Epoch 5/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - loss: 39876.0820 - val_loss: 49046.2773
Epoch 6/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - loss: 40930.4961 - val_loss: 49046.7031
Epoch 7/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - loss: 35347.0391 - val_loss: 49047.1836
Epoch 8/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - loss: 39833.8125 - val_loss: 49047.4023
Epoch 9/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - loss: 36756.8359 - val_loss: 49047.7617
Epoch 10/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - loss: 38499.4805 - val_loss: 49048.1250
Epoch 11/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - loss: 39547.2227 - val_loss: 49048.3

In [38]:
X_train_encoded = autoencoder.predict(X_mint_train)
X_test_encoded = autoencoder.predict(X_mint_test)

1/2 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/stepWARNING:tensorflow:5 out of the last 8 calls to <function TensorFlowTrainer.make_predict_function.<locals>.one_step_on_data_distributed at 0x1383462a0> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step


In [39]:
model = Model(input_dim=X_train_encoded.shape[1], output_dim=1)

In [40]:
model.fit(X_train_encoded, y_mint_train, validation_split=0.2, epochs=100, batch_size=32,
                    callbacks=[
                                EarlyStopping(monitor='val_loss',
                                             patience=5,
                                             restore_best_weights=True),
                                ReduceLROnPlateau(
                                  monitor='val_loss',
                                  factor=0.5,
                                  patience=5,
                                  verbose=1)
                               ]
                    )

Epoch 1/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 290ms/step - accuracy: 0.4744 - loss: 0.8828 - val_accuracy: 0.1818 - val_loss: 0.7439 - learning_rate: 1.0000e-06
Epoch 2/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - accuracy: 0.4081 - loss: 0.8965 - val_accuracy: 0.1818 - val_loss: 0.7416 - learning_rate: 1.0000e-06
Epoch 3/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - accuracy: 0.5208 - loss: 0.8268 - val_accuracy: 0.1818 - val_loss: 0.7389 - learning_rate: 1.0000e-06
Epoch 4/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 0.2907 - loss: 1.0152 - val_accuracy: 0.1818 - val_loss: 0.7370 - learning_rate: 1.0000e-06
Epoch 5/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - accuracy: 0.3674 - loss: 0.9628 - val_accuracy: 0.1818 - val_loss: 0.7351 - learning_rate: 1.0000e-06
Epoch 6/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - accuracy: 0.3977 - loss: 0.9059 - val_accuracy: 0.1818 - val_loss: 0.7337 - learning_rate: 1.0000e-06
Epoch 7/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - accuracy: 0.3267 - loss

In [41]:
y_pred_prob = model.predict(X_test_encoded)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step


In [42]:
best_thresholds, _ = tune_thresholds(y_mint_test, y_pred_prob)

Label 0: Best threshold = 0.00, Best f1 = 0.6000


In [43]:
y_pred = (model.predict(X_test) >= best_thresholds).astype(int)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step


### Report

In [44]:
print(classification_report(y_mint_test, y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         8
           1       0.43      1.00      0.60         6

    accuracy                           0.43        14
   macro avg       0.21      0.50      0.30        14
weighted avg       0.18      0.43      0.26        14

